In [ ]:
!pip install tensorflow

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("Roman-Urdu-Poetry.csv", encoding='utf-8')
df.head()

,ID,Poet,Poetry
0,1,ahmad-faraz,aañkh se duur na ho dil se utar jā.egā \nvaqt ...
1,2,ahmad-faraz,āshiqī meñ 'mīr' jaise ḳhvāb mat dekhā karo \n...
2,3,ahmad-faraz,ab aur kyā kisī se marāsim baḌhā.eñ ham \nye b...
3,4,ahmad-faraz,ab ke ham bichhḌe to shāyad kabhī ḳhvāboñ meñ ...
4,5,ahmad-faraz,ab ke tajdīd-e-vafā kā nahīñ imkāñ jānāñ \nyaa...


In [3]:
# extracting poetry column only
poetry_lines = df["Poetry"].dropna().tolist()
print("Poetry LInes:", poetry_lines)


text = "\n".join(poetry_lines)

Poetry LInes: ["aañkh se duur na ho dil se utar jā.egā \nvaqt kā kyā hai guzartā hai guzar jā.egā \nitnā mānūs na ho ḳhalvat-e-ġham se apnī \ntū kabhī ḳhud ko bhī dekhegā to Dar jā.egā \nDūbte Dūbte kashtī ko uchhālā de duuñ \nmaiñ nahīñ koī to sāhil pe utar jā.egā \nzindagī terī atā hai to ye jaane vaalā \nterī baḳhshish tirī dahlīz pe dhar jā.egā \nzabt lāzim hai magar dukh hai qayāmat kā 'farāz' \nzālim ab ke bhī na ro.egā to mar jā.egā", "āshiqī meñ 'mīr' jaise ḳhvāb mat dekhā karo \nbāvle ho jāoge mahtāb mat dekhā karo \njasta jasta paḌh liyā karnā mazāmīn-e-vafā \npar kitāb-e-ishq kā har baab mat dekhā karo \nis tamāshe meñ ulaT jaatī haiñ aksar kashtiyāñ \nDūbne vāloñ ko zer-e-āb mat dekhā karo \nmai-kade meñ kyā takalluf mai-kashī meñ kyā hijāb \nbazm-e-sāqī meñ adab ādāb mat dekhā karo \nham se durveshoñ ke ghar aao to yāroñ kī tarah \nhar jagah ḳhas-ḳhāna o barfāb mat dekhā karo \nmāñge-tāñge kī qabā.eñ der tak rahtī nahīñ \nyaar logoñ ke laqab-alqāb mat dekhā karo \ntishnagī

In [4]:
# tokenization
char_vocab = sorted(set(text))
vocab_size = len(char_vocab)
print("Vocabulary size:", vocab_size)
print("Vocabulary:", char_vocab)

char_to_index = {char: idx for idx, char in enumerate(char_vocab)}
print("Character to Index:", char_to_index)

index_to_char = {idx: char for idx, char in enumerate(char_vocab)}
print("Index to Character:", index_to_char)

Vocabulary size: 46
Vocabulary: ['\n', ' ', '!', "'", ',', '-', '.', '?', 'D', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ñ', 'ā', 'ē', 'ġ', 'ī', 'ū', '۔', 'Ḍ', 'ḳ']
Character to Index: {'\n': 0, ' ': 1, '!': 2, "'": 3, ',': 4, '-': 5, '.': 6, '?': 7, 'D': 8, 'H': 9, 'T': 10, 'a': 11, 'b': 12, 'c': 13, 'd': 14, 'e': 15, 'f': 16, 'g': 17, 'h': 18, 'i': 19, 'j': 20, 'k': 21, 'l': 22, 'm': 23, 'n': 24, 'o': 25, 'p': 26, 'q': 27, 'r': 28, 's': 29, 't': 30, 'u': 31, 'v': 32, 'w': 33, 'x': 34, 'y': 35, 'z': 36, 'ñ': 37, 'ā': 38, 'ē': 39, 'ġ': 40, 'ī': 41, 'ū': 42, '۔': 43, 'Ḍ': 44, 'ḳ': 45}
Index to Character: {0: '\n', 1: ' ', 2: '!', 3: "'", 4: ',', 5: '-', 6: '.', 7: '?', 8: 'D', 9: 'H', 10: 'T', 11: 'a', 12: 'b', 13: 'c', 14: 'd', 15: 'e', 16: 'f', 17: 'g', 18: 'h', 19: 'i', 20: 'j', 21: 'k', 22: 'l', 23: 'm', 24: 'n', 25: 'o', 26: 'p', 27: 'q', 28: 'r', 29: 's', 30: 't', 31: 'u', 32: 'v', 3

In [5]:
# text to numerical sequences
encoded_text = [char_to_index[c] for c in text]

# input-output sequences
seq_length = 100
sequences = []
for i in range(len(encoded_text) - seq_length):
    sequences.append(encoded_text[i: i + seq_length + 1])

sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]

# one-hot encoder
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

In [6]:
# GRU Model
model = Sequential([
    Embedding(vocab_size, 256, input_length=seq_length),
    GRU(512, return_sequences=True),
    GRU(512),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [13]:
# model training
epochs = 6
batch_size = 64
model.fit(X, y, batch_size=batch_size, epochs=epochs)

Epoch 1/6
13869/13869 ━━━━━━━━━━━━━━━━━━━━ 657s 47ms/step - accuracy: 0.5626 - loss: 1.4391
Epoch 2/6
13869/13869 ━━━━━━━━━━━━━━━━━━━━ 685s 48ms/step - accuracy: 0.5712 - loss: 1.4056
Epoch 3/6
13869/13869 ━━━━━━━━━━━━━━━━━━━━ 680s 47ms/step - accuracy: 0.5735 - loss: 1.3961
Epoch 4/6
13869/13869 ━━━━━━━━━━━━━━━━━━━━ 679s 47ms/step - accuracy: 0.5708 - loss: 1.4065
Epoch 5/6
13869/13869 ━━━━━━━━━━━━━━━━━━━━ 656s 47ms/step - accuracy: 0.5455 - loss: 1.5090
Epoch 6/6
13869/13869 ━━━━━━━━━━━━━━━━━━━━ 683s 47ms/step - accuracy: 0.3966 - loss: 2.1234


In [14]:
model.save(f"poetry_model({epochs}epoch).h5")

In [7]:
def generate_poetry(seed_text, next_chars=200):
    result = seed_text
    for _ in range(next_chars):
        encoded_input = [char_to_index[c] for c in result[-seq_length:]]
        encoded_input = np.array(encoded_input).reshape(1, -1)
        prediction = model.predict(encoded_input, verbose=0)
        next_char = index_to_char[np.argmax(prediction)]
        result += next_char
    return result

In [9]:
initial_text = "chai"
generated_poetry = generate_poetry(initial_text)
print(generated_poetry)

chain 
shāyad ki ham se gardish-e-dil ko 
maiñ ne dekh kar rahā huuñ 
maiñ ne kahā huuñ maiñ 
apnā hai maiñ ne maiñ ne dekhūñ 
ki maiñ ne maidāñ ke bahut hai maiñ 
ki mujh ko hamārā huuñ maiñ 
apnā hai ma


In [10]:
seed_text = "aañkh se duur"
generated_poetry = generate_poetry(seed_text)
print(generated_poetry)

aañkh se duur 
tarz-e-dil ko mujh ko bahā hai 
dil ko maiñ ne dekhā hai 
maiñ ne maiñ ne dekhā hai 
maiñ ne maiñ ne dil kī hai 
maiñ ne dekh kar 
kahā hai mardis meñ aaj 
ki mujh ko mujh se ham ne 
aaj hai magar h


## Loading the saved model

In [5]:
from tensorflow.keras.models import load_model

model = load_model("poetry_model(6epoch).h5")

In [8]:
# character mappings
char_vocab = sorted(set("\n !',-.?DHTabcdefghijklmnopqrstuvwxyzñāēġīū۔Ḍḳ"))
char_to_index = {char: idx for idx, char in enumerate(char_vocab)}
index_to_char = {idx: char for idx, char in enumerate(char_vocab)}

seq_length = 100

19 epoch

In [ ]:
seed_text = "aañkh se duur"
generated_poetry = generate_poetry(seed_text)
print(generated_poetry)

aañkh se duur ke haiñ 
ham kar ke meñ kar ke haiñ ke rahā 
jaa.e meñ kī mat hai meñ kar kī 
maiñ meñ maiñ meñ hai haiñ ko meñ kar kā hai 
ham hai meñ kar kar 
ye bhī hai mar ke hai tam kar ke haiñ ke 
maiñ meñ hai


6 epoch

In [11]:
seed_text = "chai"
generated_poetry = generate_poetry(seed_text)
print(generated_poetry)

chain 
shāyad ki ham se gardish-e-dil ko 
maiñ ne dekh kar rahā huuñ 
maiñ ne kahā huuñ maiñ 
apnā hai maiñ ne maiñ ne dekhūñ 
ki maiñ ne maidāñ ke bahut hai maiñ 
ki mujh ko hamārā huuñ maiñ 
apnā hai ma


3 epoch

In [ ]:
seed_text = "chai"
generated_poetry = generate_poetry(seed_text)
print(generated_poetry)

chain 
jalva-e-zabāñ kī baat kahāñ kahte haiñ 
aaj bhī haiñ jalāla se 
jalva-e-be-dil karte haiñ 
jalā ke dekhnā hai ki jo dil kā 
jalva-e-shabāñ kī baat baaqī 
jo dil kā jalāl aa.e haiñ 
jalā ke dekhnā h


In [ ]:
seed_text = "gadi kharab ae"
generated_poetry = generate_poetry(seed_text)
print(generated_poetry)

gadi kharab ae 
jo dil kī vaqt kā sabza hai jo dil kā samajh baaqī 
jo dil kī baat baaqī hai ki jo dil kā 
jalva-e-shabāñ kī baat bane dil kā 
jalva-e-shabāñ kī baat baaqī hai 
jalva-e-manzil kī baat baaqī 
jo dil 


In [ ]:
seed_text = "wo kon thi"
generated_poetry = generate_poetry(seed_text)
print(generated_poetry)

wo kon thiiñ 
ab us kī baat kahāñ kahāñ karte haiñ 
ki jo dil kā jalāl kar dekhā hai 
jalā ke dekhnā hai ki jo dil kā samajh baaqī 
aaj bhī hai ki jalā diyā āshnā 
jalva-e-shabāñ kī baat bane dil kā 
jalva-e-sh


In [ ]:
seed_text = "assignment"
generated_poetry = generate_poetry(seed_text)
print(generated_poetry)

assignmentī 
dil ko bhī kal jaanā thā 
aaj bhī hai ki jalā diyā 
jalā ke dekhnā hai ki jo dil kā 
jalva-e-shabāñ kī baat bane 
aaj bhī hai ki jalā diyā 
jalā ke dekhnā hai ki jo dil kā 
jalva-e-shabāñ kī baat b


In [ ]:
seed_text = "pyar"
generated_poetry = generate_poetry(seed_text)
print(generated_poetry)

pyar 
aaj kī baat baat hai 
jo dil kā jalāl aa.e haiñ 
jalā ke dekhnā hai ki jo dil kā 
aaj kahāñ kahte haiñ ki jo dil kā 
jalva-e-shabāñ kī baat kahāñ kahte haiñ 
jalva-e-manzil kabhī aatā hai ki jo dil 


In [ ]:
seed_text = "ab ke hum"
generated_poetry = generate_poetry(seed_text)
print(generated_poetry)

ab ke hum 
jalva-e-mahfil kī baat baaqī 
jo dil kā sabz kar de 
ai 'valī' ki jo dil kā samajh baaqī 
jab dil kā sabza hai jo dil kā 
jalva-e-shabāñ kī baat karte haiñ 
jo dil kī dil kī daaġh kā saarā hai 
jalv


In [ ]:
seed_text = "ab shauq se"
generated_poetry = generate_poetry(seed_text)
print(generated_poetry)

ab shauq se 
jalva-e-zabāñ kī baat karte haiñ 
jab ki dil kī baat baaqī hai 
jalva-e-manzil kī baat baaqī 
jo dil kī jalā ke baḌe dil kā 
jalva-e-shabāñ kī baat bane 
aaj bhī hai ki jalā diyā 
jalā ke dekhnā hai


In [ ]:
seed_text = "dost ban kr"
generated_poetry = generate_poetry(seed_text)
print(generated_poetry)

dost ban krā 
aaj baaqī hai ki jo dil kā samajh baaqī 
jo dil kī dard kī baat baat hai 
jo dil kā jalāl aa.e haiñ ki jin kā saarā hai 
jalva-e-zabāñ kī baat kahāñ kahāñ kahte haiñ 
aaj bhī haiñ jalāla kī baat ka


In [ ]:
seed_text = "woh kon thi"
generated_poetry = generate_poetry(seed_text)
print(generated_poetry)

woh kon thiiñ 
jo dil kī baat baat bane dil kā 
jalva-e-shabāñ kī baat baaqī hai 
jalva-e-manzil kī baat baat bane 
aaj bhī aatā hai ki jo dil kā 
jalva-e-shabāñ kī baat baaqī 
jab dil ko bhī nahīñ kisī kā 
jalv


In [ ]:
seed_text = "porsche"
generated_poetry = generate_poetry(seed_text)
print(generated_poetry)

porsche 
jo dil kā jalāl aa.e haiñ 
jalā ke dekhnā hai ki jo dil kā 
aaj kahāñ kahte haiñ 
jalā ke paas haiñ jalā ke rahā 
aaj bhī haiñ jalālā ke dekhte haiñ 
ab ke baaqī hai ki jo dil kā 
jalva-e-shabāñ ke 


In [ ]:
seed_text = "meri porsche"
generated_poetry = generate_poetry(seed_text)
print(generated_poetry)

meri porsche 
jo dil kī vaqt kā sabza hai jo dil kā 
jalva-e-shabāñ kī baat kahāñ kahte haiñ 
jo dil kā jalāl kar dekhā hai ki jo dil kā 
jalva-e-shabāñ kī baat baaqī hai 
jalva-e-manzil kī baat banā diyā 
jo dil
